In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.
c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\tensorflow\python\framework\dtypes.py:472: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\tensorflow\python\framework\dtypes.py:473: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 =

In [3]:

with open('dataset_pozytywizm.txt','r',encoding='utf-8') as txt:
    text = txt.read()
print('corpus length:', len(text))

corpus length: 112829


In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 111


In [5]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 37597


In [6]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [7]:
y.shape

(37597, 111)

In [8]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))


Build model...


In [9]:
# # build the model: a single LSTM
# print('Build model...')
# model = Sequential()
# # model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# model.add(Dense(len(chars), activation='softmax', input_shape=(maxlen, len(chars))))
# model.add(LSTM(len(chars)))
# model.add(Dense(len(chars), activation='softmax'))


In [10]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [11]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

Epoch 1/20
37504/37597 [============================>.] - ETA: 0s - loss: 2.8280
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "
Jeśli nie doznasz nigdy zawodów
W tem, "

Jeśli nie doznasz nigdy zawodów
W tem, zarały się wieliem porzeje,
Be wierze się naszarzem podzierzem przewielie,
I wiesze wierze wierzem zarzemiem wierzem wierzem wierzewiem wierzech przewierzem swole,
Na się wierzenieje się naszycha wierzemień wierzech wierzem wierzem wierzem wierzem wielie,
Z się wierze wierzych wierzeszanie porowień warzegie,
I się podzie wierzem podzierzewiem wierzych przewiemie,
Na wierze w zierzecze wierzem powi
----- diversity: 0.5
----- Generating with seed: "
Jeśli nie doznasz nigdy zawodów
W tem, "

Jeśli nie doznasz nigdy zawodów
W tem, te wiebyie powiele,
I zieli nam warzeszeczycha najnej,
I nie wieczuty wihwierzyczych wioszemią waszczeły,
Z i nie wedzychością gorowiech swierwień potawioty
Z brzeczech zsprzeniemi,
W nie zwierze wierto,
Na się wierze

Słonecz w mdysi weci naski,
Czy sierzwa ząstopacie ródrzeńcy,

Otym przed zworajech szczecie?
ICz łęczy tadno żawnę róm oełocy!
Nielak, o z w pórsiona totniwa,
  Jażdo lazbówiłi ujuci,
zy płłnograchota stum, sercia.


Sdnda buctemzki i pałąch złus.

Bolam badto na swój od niżdylstów,
I ciebię! wipdróż bymą nie noje
I serca dlujtą, piecść słyma.

Za tońcy kostany nie mylaki kawieniędy,
I krzyty ech towłąc
----- diversity: 1.2
----- Generating with seed: "ytu!

Zatoczcie nad naszą ziemią
Słonecz"
ytu!

Zatoczcie nad naszą ziemią
Słoneczna Panke możka serzym skarzeł botata!

Jużemi odliwa się ołhkętąca,
Skod Bez, tan tren orkas

Aczne do żdzieś wawagołąlekiąłimy zawitatu,
A jedna Jestem, Sto do kań
Piękniycmego wojniem płaczy,
Już sze lboni kadrana cheja,
Wsolndych, nie skarłem, bilki ko ciwe tako,
A rły, BojByś jamuśnie jaki*,

ymy, wsię Posęch w śniczy bez tamie,
Zjeste meko, w jesk do daszka stuły
šchu bikra, gdy cała… Słóżkł 
37597/37597 [==============================] - 33s 867us/st





Adam Asnyk

Twe nas ten dale














Adam Asnyk

Tak tak stary
Na te w dom stoi tak droga,
A z pod wiedzie się stary,
W cierkiem na stoi tak praca,
I nie wsparzeń się się stoi tam pracę,
I nie wody stary, tak do nam pod polemie,
Więc pod potacha, jak martarzych promienie
Na miłości tań moje z jakiem przestanie,
I z dobraczy się w polek stopodzi,
I z dom szczęść nie wielkiem się w poliem
----- diversity: 0.5
----- Generating with seed: "li większe plamy,
I tak sobie pomagamy.
"
li większe plamy,
I tak sobie pomagamy.





Adam Asnyk

Za siedzi za marzeń taż doszekoty
I stoich zamarzeń swoje na wszystej to wodzie



Wastrga mało króle,
W do niech strach tam młodu w polekowa





Nad tanek

Więc pod płanków skrzydziejem do podzienia
Oniło za umienie poszcze,
    Do bogo, którem się w kary,
W potrzy się na ten,
A tak nas tam tylko, o tak czercielie,
I wszystkie swoje z jad polskie sprycha,





Adam Asnyk

Do tak
----- diversity: 1.0
----- Generating with seed: "li większe plamy,

Da chorza nieść chybie szcha się dlejkę
zona cię wiesz! przelatamł łena,
  Zapierzednie po jawej policza.

Dziktnie ichrianyków swejona ziodzie
wozaci pochł świat wierzej kochaszłośli wiosna
Zmamając gazana,
W wchową nie radest z godzia!

Na co śpiesły dłaszce,
ech żytać w świat krośn
----- diversity: 1.2
----- Generating with seed: "i chwale,

Wtenczas pierś każda ludzka o"
i chwale,

Wtenczas pierś każda ludzka ołębie!a.

Gtał tas teetem złoznów słupnącza święty podełdałęga,
Azy sercu z świat nie bfachgi pieszczą goły.

Marma nic był Budzą erocą.

Jakżu pobuj dzistelmów Bóg jużówie.

Lecw supiedzi wzków sichotej niesze fiery.

Kwódd, primnysbem Niechowa!



Adam Asnyk

Aż czasz
! boty Brask skun na tesok miro.

Kawia jakłe musku żadką,
Z niwa zbieckią duch…
Bora to postaćka, wiezmę towali znąc zosła;
Sobi
37597/37597 [==============================] - 33s 884us/step - loss: 1.4891
Epoch 12/20
37504/37597 [============================>.] - ETA: 0s - loss: 1.4449
----- Generating text 


































Adam Asnek

Dwielki







Adam Asnyk

Dwieli.







Adam Asnyk

Doten,







Adam Asnyk

Dwieli.







Adam Asnyk

Dotych pochcaną

Tam poszlał na stała,
Który w pierść nie wiecza nie szłomy,
I nie wielki się zasłodych poczczy kochatni
Skajmy wiosny kociata,
Jak się jedza posłada w przydubie,
W niw pomali 
----- diversity: 0.5
----- Generating with seed: "cem;
Lecz nie było tam słowa nadziei.
Po"
cem;
Lecz nie było tam słowa nadziei.
Podnik






















Adam Asnek

Ktorej to miłości i wiekie.




Adam Asnyk

Za Kudy:
Wteko słody się się zago mało.




Adam Asnyk

Dotowa







Adam Asnyk

Za Cój pierzcie,
Jak nie może się w niebiosne i kochali
Za wież — podnie zakomienia,
I stróż narosi przeciedzie
I nigdziętnie truc ducha, ani strostejskiem,
To mocy nie miecie,
Aż poszczy znowu i w choć kochany,

Tak nie miecie zgorze 
----- diversity: 1.0
----- Generating with seed: "cem;
Lecz nie było tam słowa nadziei.
Po"
cem;
Lecz nie było tam słowa n

A zadala mias, w dobrzego seł rzewesza do komitnia!



VIra!

Piiliios Jawez kwiecg pokoczy buszemski świętej sobie z zostało!
I białą go ciło zacłasne idzie,
Awe z pamięnem — przeraz Janeki
la łódy zachołe?
ąppikrzy rzuleć nie pomataną w zarapych lab
----- diversity: 1.2
----- Generating with seed: "ń godzi.

Bezsilne gniewy, zasadzki i zd"
ń godzi.

Bezsilne gniewy, zasadzki i zdobęędarkica,
uprzepujem grzęci, masz! któreszka gię,y, co tak,
C, tają już nie gdzieś chary,
W blas żyk go na ślata,
Oni snołem w zostadują stary,
Onekbie Zmuknie spuczy…

Bigdvie! biał!cks
Tukiesz shlebany…



Numiem lego dzieckich —
Róż tosłym dalej:
Tor na soceko!
A co kołania, przeroka, co w piść nie bielaszko,
  bitła pukom żyki sokre soko
Oni krużgi, pragniadźmierówa.

Beziem pyłach w — bie.
37597/37597 [==============================] - 34s 909us/step - loss: 1.2786
Epoch 19/20
37504/37597 [============================>.] - ETA: 0s - loss: 1.2533
----- Generating text after Epoch: 18
----- diversity: 0

In [12]:

neural_texts = list()
fan_fic_len = 400

for n in range(10):
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.7]:

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence

        for i in range(fan_fic_len):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    neural_texts.append(generated)

In [13]:
neural_texts

['ieńczon różami w flet zawzięcie dmucha,\nPojatrzy ho to miłości gromą,\nCo tebrzek był swej tysie,\n    juk polekaje i była,\nA ten zapamienie zakroba,\n\n— «Wylks turędzimów swoich ramię truchą troci zaczowa.\n\n\n\nAdam Asdyk\n\nJedronki modą nie zakół\nZomeczego pokał kte kwytkiem nasze z zorzy z tenie,\nBo ot wychniej gasteńce i do błęgi,\njak żebrzeń polski, i niegdyś do brzepiony,\n  Wiosen wyciądziemh prawić się żywała,\nNa to liś, kroię duch pie',
 'j pracę naszą czuje\nTa ziemia kochana!\nWieczowa wieczek, gdzie wycho tych, Tatanne tę zdo mory.\n\nW cały widzia widział, w jako czarno,\nBo ta nie panie poszedłagiem.\n\nPrzestane wiła kochatarz i życzoł krejstej polatem,\nOddyż podzielach leczmość dziejszy i wierzchni,\nA z piersia z już dzieci,\nI poszła jesze słonki nie na światy tąd.\n\n\n\nAdam Asi kais terappntopsko?\nPtaszknów wiosnę tylko wydnowy i wiady,\nI nadwiejmi w płacić biega,\nLecz wi',
 'tem?\n\nO wielki poemacie! ciebie tylko mojejąc ganącie,\n    To sie mama 

In [14]:
model.save('.\model2.h5')
model.save_weights('.\model2Weights.h5')